#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter CM Report Replicate Parameters
Replicate a report across multiple networks and advertisers.
 1. Provide the name or ID of an existing report.
 1. Run the recipe once to generate the input sheet called CM Replicate For UNDEFINED.
 1. Enter network and advertiser ids to replicate the report.
 1. Data will be written to BigQuery &gt; UNDEFINED &gt; UNDEFINED &gt; [REPORT NAME]_All
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_read': 'user',  # Credentials used for reading data.
  'recipe_name': '',  # Sheet to read ids from.
  'auth_write': 'service',  # Credentials used for writing data.
  'account': '',  # CM network id.
  'recipe_slug': '',
  'report_id': '',  # CM template report id, for template
  'report_name': '',  # CM template report name, empty if using id instead.
  'delete': False,  # Use only to reset the reports if setup changes.
  'Aggregate': False,  # Append report data to existing table, requires Date column.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute CM Report Replicate
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'drive': {
      'auth': 'user',
      'copy': {
        'source': 'https://docs.google.com/spreadsheets/d/1Su3t2YUWV_GG9RD63Wa3GNANmQZswTHstFY6aDPm6qE/',
        'destination': {'field': {'name': 'recipe_name','prefix': 'CM Replicate For ','kind': 'string','order': 1,'description': 'Name of document to deploy to.','default': ''}}
      }
    }
  },
  {
    'dataset': {
      'auth': 'user',
      'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 2,'default': '','description': 'Name of Google BigQuery dataset to create.'}}
    }
  },
  {
    'dcm_replicate': {
      'auth': 'user',
      'report': {
        'account': {'field': {'name': 'account','kind': 'integer','order': 3,'default': '','description': 'CM network id.'}},
        'id': {'field': {'name': 'report_id','kind': 'integer','order': 4,'default': '','description': 'CM template report id, for template'}},
        'name': {'field': {'name': 'report_name','kind': 'string','order': 5,'default': '','description': 'CM template report name, empty if using id instead.'}},
        'delete': {'field': {'name': 'delete','kind': 'boolean','order': 6,'default': False,'description': 'Use only to reset the reports if setup changes.'}}
      },
      'in': {
        'sheet': {'field': {'name': 'recipe_name','prefix': 'CM Replicate For ','kind': 'string','order': 1,'default': '','description': 'Sheet to read ids from.'}},
        'tab': 'Accounts'
      },
      'out': {
        'bigquery': {
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': ''}},
          'is_incremental_load': {'field': {'name': 'Aggregate','kind': 'boolean','order': 7,'default': False,'description': 'Append report data to existing table, requires Date column.'}}
        }
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
